In [ ]:
!pip install -q langchain langchain-community langchain-huggingface chromadb langchain-core
!pip install -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 21.2 MB/s eta 0:0

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from google.colab import drive

In [ ]:
import json
import os
import re
import shutil

In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from langchain_core import vectorstores
data_cleaned_dir = "/content/drive/MyDrive/chatbot_project/data_cleaned"
embedding_model_path = "/content/drive/MyDrive/chatbot_project/models/embedding"
vector_db_dir = "/content/drive/MyDrive/chatbot_project/vectorstore/chroma_db"


Lấy mã nguồn từ file "ETL" để thực hiện tạo các chunk

In [ ]:
# 1. Định nghĩa cấu hình cắt
headers_to_split_on = [
    ("#", "chapter_title"),  # Cấp 1: Chương
    ("##", "article_title"), # Cấp 2: Điều
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Bộ cắt phụ: Chỉ dùng khi 1 Điều quá dài (VD: Điều 12 dài > 2000 ký tự)
text_recursive = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
def split_text_keeping_tables(text):
    # Regex tìm bảng Markdown:
    # Bắt đầu bằng |...|, dòng tiếp theo |---|...|, và các dòng tiếp theo bắt đầu bằng |
    # (?m) bật chế độ multiline
    table_pattern = r'(?:\|.*\|\n\|[-:| ]+\|\n(?:\|.*\|\n?)*)'

    # Tìm tất cả các bảng
    tables = list(re.finditer(table_pattern, text))

    if not tables:
        # Nếu không có bảng, cắt bình thường
        return text_recursive.split_text(text)

    final_chunks = []
    last_end = 0

    for match in tables:
        start, end = match.span()

        # 1. XỬ LÝ PHẦN VĂN BẢN TRƯỚC BẢNG
        pre_table_text = text[last_end:start]
        if pre_table_text.strip():
            final_chunks.extend(text_recursive.split_text(pre_table_text))

        # 2. XỬ LÝ PHẦN BẢNG (GIỮ NGUYÊN)
        table_text = match.group()
        if len(table_text) > 4000:
             final_chunks.append(table_text)
        else:
             final_chunks.append(table_text)
        last_end = end

    # 3. XỬ LÝ PHẦN VĂN BẢN CÒN LẠI SAU BẢNG CUỐI
    post_table_text = text[last_end:]
    if post_table_text.strip():
        final_chunks.extend(text_recursive.split_text(post_table_text))

    return final_chunks

In [ ]:
def chunk_json_file(json_path):
# Đọc file JSON
  with open(json_path, 'r', encoding='utf-8') as f:
      data = json.load(f)

  # Lấy Metadata gốc (Cực kỳ quan trọng)
  base_metadata = data['metadata']
  content = data['content']

  # BƯỚC 1: Cắt theo cấu trúc Markdown (Chương/Điều)
  md_header_splits = markdown_splitter.split_text(content)
  final_chunks = []

  # BƯỚC 2: Kiểm tra độ dài và Gắn Metadata gốc
  for split in md_header_splits:
    content_of_article = split.page_content
    metadata_of_article = split.metadata
    sub_chunks = split_text_keeping_tables(content_of_article)
    for chunk_text in sub_chunks:
          # Kiểm tra xem chunk này có phải là bảng không để gán nhãn (Optional)
          is_table = chunk_text.strip().startswith("|")

          combined_metadata = {
              **base_metadata,
              **metadata_of_article,
              "is_table": is_table # Thêm cờ này để sau này dễ lọc
          }

          doc = Document(
              page_content=chunk_text,
              metadata=combined_metadata
          )
          final_chunks.append(doc)
  return final_chunks

In [ ]:
files = [f for f in os.listdir(data_cleaned_dir) if f.endswith('.json')]
all_chunk = []
for filename in files:
    file_path = os.path.join(data_cleaned_dir, filename)
    print(f"Processing: {filename}")
    file_docs = chunk_json_file(file_path)
    all_chunk.extend(file_docs)
print(f"FINISHED with {len(all_chunk)}")

Processing: Quy_che_25.json
Processing: QD_NN_K68.json
Processing: HD_hoc_chuyen_tiep_ky_su_180TC.json
Processing: QD_ban_hanh_QD_to_chuc_thi_Truc_tuyen.json
Processing: QD_chuyen_doi_hoc_phan_tuong_duong.json
Processing: Quy_che_CTSV_ĐHBK_HN_2025310_final.json
Processing: Hoc_bong_KKHT_2023.json
Processing: Hoc_bong_TDN_2023.json
Processing: QD_NN_K70.json
Processing: QD_NN_K65.json
Processing: QD_ban_hanh_QD_to_chuc_day_hoc_tren_nen_tang_CN_ket_noi.json
FINISHED with 489


2. Thực hiện Embedding

In [ ]:
def create_vector_db():
    # 1. Chuẩn bị Documents
    documents = all_chunk

    if not documents:
        print("Không có dữ liệu để xử lý.")
        return
    print(f"\nĐang tải model embedding: {embedding_model_path}...")
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_path,
        model_kwargs={'device': 'cuda', 'trust_remote_code': True}, # Dùng GPU nếu có
        encode_kwargs={'normalize_embeddings': True} # GTE cần normalize
    )

    print(f"Đang tạo Vector Database tại: {vector_db_dir}")

    # Xóa DB cũ nếu muốn làm mới hoàn toàn (Cẩn thận!)
    if os.path.exists(vector_db_dir):
        shutil.rmtree(vector_db_dir)

    # Tạo DB và nạp dữ liệu (Batch processing tự động)
    # Chroma tự động lưu xuống ổ cứng
    vector_db = Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        persist_directory=vector_db_dir
    )

    print("✅ Đã lưu Vector Database thành công!")
    return vector_db

In [ ]:
db = create_vector_db()


Đang tải model embedding: /content/drive/MyDrive/chatbot_project/models/embedding...


The tokenizer you are loading from '/content/drive/MyDrive/chatbot_project/models/embedding' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Đang tạo Vector Database tại: /content/drive/MyDrive/chatbot_project/vectorstore/chroma_db
✅ Đã lưu Vector Database thành công!


In [ ]:



print("\n--- [TEST TRUY VẤN VỚI ĐIỂM SỐ] ---")
query = "Điều kiện nhận học bổng loại A"

# Sử dụng similarity_search_with_score thay vì similarity_search thường
# Kết quả trả về là một list các tuple: (Document, Score)
results_with_score = db.similarity_search_with_score(query, k=3)

for i, (doc, score) in enumerate(results_with_score):
    similarity_percent = (1 - score) * 100
    if similarity_percent < 0: similarity_percent = 0

    print(f"\n✅ KẾT QUẢ {i+1}:")
    print(f"   📄 Nguồn: {doc.metadata.get('source_filename')} - {doc.metadata.get('article_title')}")
    print(f"   📏 Khoảng cách (Distance): {score:.4f} (Càng nhỏ càng tốt)")
    print(f"   🎯 Độ tương đồng ước lượng: {similarity_percent:.2f}%")
    print(f"   📝 Nội dung trích dẫn: \"{doc.page_content[:200]}...\"")


--- [TEST TRUY VẤN VỚI ĐIỂM SỐ] ---

✅ KẾT QUẢ 1:
   📄 Nguồn: Hoc_bong_KKHT_2023.pdf - Điều 3. Tiêu chuẩn xét cấp học bổng
   📏 Khoảng cách (Distance): 0.4699 (Càng nhỏ càng tốt)
   🎯 Độ tương đồng ước lượng: 53.01%
   📝 Nội dung trích dẫn: "1. Học bổng loại C: GPA ≥ 2,5 và điểm rèn luyện học kỳ ≥ 65 điểm.
2. Học bổng loại B: GPA ≥ 3,2 và điểm rèn luyện học kỳ ≥ 80 điểm.
3. Học bổng loại A: GPA ≥ 3,6 và điểm rèn luyện học kỳ ≥ 90 điểm...."

✅ KẾT QUẢ 2:
   📄 Nguồn: Hoc_bong_TDN_2023.pdf - Điều 6. Nguyên tắc xét, cấp Học bổng Trần Đại Nghĩa
   📏 Khoảng cách (Distance): 0.4901 (Càng nhỏ càng tốt)
   🎯 Độ tương đồng ước lượng: 50.99%
   📝 Nội dung trích dẫn: "1. Học bổng Trần Đại Nghĩa được xét, cấp theo học kỳ đối với các trường hợp
quy định tại khoản 1 và khoản 2 Điều 5; xét cấp thường xuyên đối với các trường hợp
quy định tại khoản 3 và khoản 4 Điều 5.
..."

✅ KẾT QUẢ 3:
   📄 Nguồn: Hoc_bong_KKHT_2023.pdf - Điều 4. Nguyên tắc xét cấp học bổng
   📏 Khoảng cách (Distance): 0.5034 (Càng 